<h2 style="background-color:powderblue;color:brown;"> <center> 20th Tropical Cyclones and Storm Surge Forecasters Training, RSMC, New Delhi </center> </h2> <hr>
<h1 style="color:tomato;"> <center>  Tropical Cyclone Features Analysis </center> </h1>
<h3 style="color:Blue;"><center>  Discussion Material with codes [21st August 2024 06.30-08.00 UTC]</center> </h3>
<h4 style="color:Purple;"> <center>B. Arul Malar Kannan,Scientist IMD </center> </h4> <h5><center> ba.kannan@imd.gov.in </center><hr></h5> <p>We all know,  Weather radar gives more important information of near real time events and is the best tool for operational nowcasting by profiling/scanning the surrounding hemispherical atmosphere around the radar center by actively sending a microwave signal and processing the backscattering signals.</p><p>  The meteorological radars are primarily aimed for detecting precipitation and related phenomena.  However, by virtue signatures of all objects present around are invariably present. </p>
<p>  The present discussion is aimed to analyze the acquired radar data with specific reference for Tropical cyclone features, a continuation of the earlier 19th Training Workshop.</p>
<p>  Open source Pythonic radar libraries ARM-PyART, X-Radar and its dependencies are extensively used enabling the user to correlate and analyse the previous theoritical discussion session 0500-0600UTC. The theoratical session material/ presentations are shared through the organizers.</p>
<p></p>
<p>  <h5> The customized code and data is shared for research purposes. While reusing the data India Meteorological Department, MoES shall be credited </h5></p>

### STEP - 1 : Import required PYTHON Libraries

In [ ]:
#IMPORT REQUIRED PYTHON LIBRARIES
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pyart
import numpy as np
import xarray as xr
import xradar as xd
import glob
import hvplot.xarray
import holoviews as hv
import panel as pn
hv.extension("bokeh") #  matplotlib
%matplotlib inline  

### Step 2: Read the Radar Data and understanding its contents

In [ ]:
#READ THE FILE and get its contents using pyART
InputFile='data/IMDTEST200602235215'
radar=pyart.io.read(InputFile)
radar.info()

In [ ]:
#READ THE FILE and get its contents using X-Radar
radar_int = xd.io.open_iris_datatree(InputFile).xradar.georeference() #dtree = xd.io.open_iris_datatree(InputFile)
radar_int #display(dtree)

<head>
    <h1>RAW DATA - The Usable Level II</h1>
    </head>
    <p>
    The pulse radar gets a basic resolution along the scanned range normally to the measure of the pulse width.  This is normally 150m [for 1micro seconds PW] and 300m[for a 2micro second PW].  The the angular resolution is approximately $1^0$ x $1^0$.  This base resolution of the entire volume is called a range bin.  All these range bins stacked along a particular direction of azimuth or elevation is termed as a ray, a collection of rays constitutes a sweep.  The sweep along a particular fixed elevation angle is called a PPI scan and the one with a fixed azimuth is termed as a RHI scan. The collection of these sweeps constitutes a volume scan.
    IMD radars perform two types of scan every ten minutes.  One for long range surveillance where the  reflectivity field is alone used and another volumetric data with ten sweeps, elevation angles 0.2, 1.5, 2.0, 3.0, 4.5, 6.0, 9.0, 12.0, 16.0, 21.0,  scan range 250km useful for weather study with all basic moments viz., reflectivity, velocity, spectrum width availability [SK Roy Bhowmick etal 2011]. 
    Present Day Dual Polarization techniques provides additional information or moments determined from the target namely, Differential reflectivity, Differential Phase, Correlation coefficient.
   </p>

<head>
    <h2>RADAR - Data storage and evolving evolving uniform Formats</h2>
    </head>
   <p> CF Conventions & Hirearchial Formats</p>


In [ ]:
ds = xr.open_dataset(InputFile, engine="iris", group="sweep_0")
display(ds)

In [ ]:
ds.azimuth.plot(y="time")

In [ ]:
ds.DBZH.sortby("time").plot(y="time")

In [ ]:
ds.DBZH.plot(y="azimuth")

In [ ]:
ho=radar.altitude['data']/1000
Rmax=int(np.floor(np.mean(radar.instrument_parameters['unambiguous_range']['data'])/1000))
BW=radar.instrument_parameters['radar_beam_width_h']['data']
Elev=(radar.elevation['data'])[0::360]
print(ho,Rmax,BW, Elev)

<head>
    <h2>RADAR - Range Height Equation and Range Height Diagram of the data</h2>
    </head>
   <p> 
    Height of the radar beam at a slant range R from the radar location at a elevation angle $\phi$ is given by<blockquote><div>
    <strong>$h_R = \sqrt{R^2+(k_e R)^2+2 k_e R\sin\phi}- k_e R + h_0$ </strong><br /></div></blockquote>
  where,<br />
    	$k_e = 4/3$    <em>for a 4/3 effective earth radius model</em> <br />
    	$\phi$ is the elevation angle <br />
    	$h_0$ is the altitude (till the center of the radiating antenna feed)<br />
    </p>
    <img src="pic/picture1.png", ,width=50%,height=50%>

In [ ]:
eee=Elev

print("Elevation angles in the present data is:", eee)
R=6371.0
k=4./3.
BW=1.0
elev=0.0
Rbin=0.5
def f(ra,p):
    return (((ra)**2.0)+((k*R)**2.0)+2*k*R*ra*np.sin(p))**0.5-k*R+ho


def rhii(elev,coo):
    phi=elev*np.pi/180.
    phi_ = np.radians(elev+BW/2.0)
    phi___ =np.radians(elev-BW/2.0)
    r=np.arange(0,Rmax,Rbin)
    h=np.zeros(len(r))
    h_ =np.zeros(len(r))
    h___ =np.zeros(len(r))
    for iii in np.arange(len(r)):
        h[iii]=f(r[iii],phi)
        h_[iii]=f(r[iii],phi_)
        h___[iii]=f(r[iii],phi___)
        bb=np.pi*(((h_-h___)/2)**2.)*r*20
    er=h_-h___
    ax.plot(r,h,linestyle='-', linewidth=2, marker='o',markersize=.2,color=coo, label=elev)
    ax.plot(r,h_,linestyle='-.', linewidth=1,color=coo) 
    ax.plot(r,h___,linestyle='-.', linewidth=1,color=coo) 
    ax.fill_between(r,h_, h___, color=coo, alpha=0.5 )
    

fig, ax = plt.subplots(figsize=(25,6))

coo=['red','green','blue','black','cyan','orange','magenta','yellow','brown','purple','gray','red','green','blue','cyan','k']
for ii in np.arange(len(eee)):
    rhii(eee[ii], coo[ii])
ax.set_xlabel('Range in km')
ax.set_ylabel('Height above radar in km')
ax.set_title('Radar Range height diagram')
ax.set_ylim([-1,20])
ax.set_xlim([0,Rmax])
minor_ticks=np.arange(0,10,0.5)
major_ticks=np.arange(0,10,1)

ax.set_yticks(minor_ticks, minor=True)
ax.grid(which='minor', alpha=0.4)
ax.grid()
#plt.show()

<head>
    <h4>Assignment: <p>1. From which start range to what end range hydrological products of radar are useful </p>
    <p> 2. When the maximum cloud detection is feasible with negative elevation angles why there is a restriction... Any comments? </p></h4>
    </head> 

<p>
Irrespective of the above calculation for the beam propagation still artifact lingers out of the anamolous propagation conditions where the refractive index variations are not to the normal way leading to sub/super refractions
    </p>
  <div class="left">
       <img src="pic/picture2.png", ,width=50%,height=50%/><br></a>
  </div>

In [ ]:
#GET A GIST ON RADAR RECORDED VARIABLES & VALUES as Key Dictionary pair
radar.fields

### Step 3: Create a PPI Plot for the observed moments

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=(25,6))
ax1 = fig.add_subplot(131)
display.plot_ppi('total_power', 0, ax=ax1, vmin=-32, vmax=64., title="Reflectivity")
ax2 = fig.add_subplot(132)
display.plot_ppi('velocity', 0, ax=ax2,  title="Velocity")
ax3 = fig.add_subplot(133)
display.plot_ppi('spectrum_width', 0, ax=ax3,vmin=0,vmax=12,  title="spectrum_width")

<head><h4>Assignment: <p>3. Comment on the obsertvations of the above plot </p></h4>  </head>
<p> Hint:  Severe Weather Signatures, Wind...  Also refere to RHI plots below</p>

In [ ]:
AZI=313
fig = plt.figure(figsize=(20,6))
ax1 = fig.add_subplot(121)
display.plot_azimuth_to_rhi('total_power', AZI, ax=ax1, vmin=-32, vmax=64., title="RHI-Reflectivity")
plt.ylim([0, 20]);
ax2 = fig.add_subplot(122)
display.plot_azimuth_to_rhi('velocity', AZI, ax=ax2, vmin=-47, vmax=47., title="RHI-Velocity")
plt.ylim([0, 20]);

### Step 4: TIME SERIES ANALYSIS

In [ ]:
InputFile1='data/IMDTEST190502235218'
InputFile2='data/IMDTEST190503060219'
InputFile3='data/IMDTEST190503115218'
radar1=pyart.io.read(InputFile1)
radar2=pyart.io.read(InputFile2)
radar3=pyart.io.read(InputFile3)
display1 = pyart.graph.RadarDisplay(radar1)
display2 = pyart.graph.RadarDisplay(radar2)
display3 = pyart.graph.RadarDisplay(radar3)
fig = plt.figure(figsize=(30,18))
ax1 = fig.add_subplot(331)
display1.plot_ppi('total_power', 0, ax=ax1, cmap='pyart_BuDRd18', vmin=0, vmax=64.,colorbar_label="Ze (dBZ)", title="Reflectivity Factor")
ax2 = fig.add_subplot(334)
display1.plot_ppi('velocity', 0, ax=ax2, cmap='pyart_BuDRd18', colorbar_label="Radial Vel (m/s)", title="Velocity")
ax3 = fig.add_subplot(337)
display1.plot_ppi('spectrum_width', 0, ax=ax3,cmap='afmhot_r',vmin=0,vmax=6,colorbar_label="W(m/s)",  title="spectrum_width")
ax4 = fig.add_subplot(332)
display2.plot_ppi('total_power', 0, ax=ax4, cmap='pyart_BuDRd18',vmin=0, vmax=64.,colorbar_label="Ze (dBZ)", title="Reflectivity Factor")
ax5 = fig.add_subplot(335)
display2.plot_ppi('velocity', 0, ax=ax5,colorbar_label="Radial Vel (m/s)",  title="Velocity")
ax6 = fig.add_subplot(338)
display2.plot_ppi('spectrum_width', 0, ax=ax6,cmap='afmhot_r',vmin=0,vmax=6,colorbar_label="W(m/s)",  title="spectrum_width")
ax7 = fig.add_subplot(333)
display3.plot_ppi('total_power', 0, ax=ax7, cmap='pyart_BuDRd18',vmin=0, vmax=64.,colorbar_label="Ze (dBZ)", title="Reflectivity Factor")
ax8 = fig.add_subplot(336)
display3.plot_ppi('velocity', 0, ax=ax8,colorbar_label="Radial Vel (m/s)",  title="Velocity")
ax9 = fig.add_subplot(339)
display3.plot_ppi('spectrum_width', 0, ax=ax9,cmap='afmhot_r',vmin=0,vmax=6,colorbar_label="W(m/s)",  title="spectrum_width")

<head><h4>Assignment: <p>4. Comment on the obsertvations of the above plot </p> <p>5. Why the System shows sudden variations?</p></h4>  </head>
<p> Hint:  Recurvature, Shear, Reorganization support features, Wind analysis...  </p>

In [ ]:
# Loop on all sweeps and compute VAD
zlevels = np.arange(100, 7000, 100)  # height above radar
u_allsweeps = []
v_allsweeps = []
RDR=radar3
for idx in range(RDR.nsweeps):
    radar_1sweep = RDR.extract_sweeps([idx]);
    vad = pyart.retrieve.vad_browning(radar_1sweep, "velocity", z_want=zlevels);
    u_allsweeps.append(vad.u_wind);
    v_allsweeps.append(vad.v_wind);

# Average U and V over all sweeps and compute magnitude and angle
u_avg = np.nanmean(np.array(u_allsweeps), axis=0);
v_avg = np.nanmean(np.array(v_allsweeps), axis=0);
orientation = (np.rad2deg(np.arctan2(-u_avg, -v_avg))) % 360;#+180
speed = np.sqrt(u_avg**2 + v_avg**2);

# Display vertical profile of wind
fig, ax = plt.subplots(1, 3, sharey=True,figsize=[5,7]);
ax[0].plot(speed * 2, zlevels + RDR.altitude["data"]);
ax[1].plot(orientation, zlevels + RDR.altitude["data"]);
ax[0].set_xlabel("Wind speed [m/s]");
ax[1].set_xlabel("Wind direction [deg]");
ax[0].set_ylabel("Altitude [m]");
fig.suptitle("VAD Wind profile"+"\n"+RDR.time['units'][-20:])
u = (np.ma.masked_array(u_avg, mask=False))*1.94364;
v = (np.ma.masked_array(v_avg, mask=False))*1.94364;
nbarbs = (~u.mask).sum();
#skip = max(1, int(nbarbs//5));
skip = max(1, int(nbarbs//10));
print(skip)
bloc = 0.5;
ax[2].barbs((np.zeros(zlevels.shape)+bloc)[::skip]-0.5, zlevels[::skip]+RDR.altitude['data'],u[::skip], v[::skip],);
ax[2].axis("off");

<head><h4>Assignment: <p>6. Comment on the wind direction variation </p> </h4>  </head>
<p> Hint:  True wind and Radial wind...  </p>

### Step 5: Data Cartesian gridding and creating Georeferenced plots

In [ ]:
#Filter and Cartesian GRID the radar data for further processing
gatefilter = pyart.filters.GateFilter(radar1)
gatefilter.exclude_below('total_power', -5) #-5dBZ
gatefilter.exclude_below('reflectivity', -5) #-5dBZ

#Grid Voxel count are 200 x 200 x 40 (X,Y,Z)
#That covers -250km to +250km x -250km to +250km x 0km to 20 km range around the radar
grid = pyart.map.grid_from_radars(radar1, grid_shape=(41, 201, 201),
                                  grid_limits=((0.,20000,), (-250000., 250000.), (-250000, 250000.)))
#Resolution vertical:500m horizontal2500m

In [ ]:
#PLOT THE GRID GEO-REFERRED
#create a figure dimension
fig = plt.figure(figsize=[10, 6])
#Create a display variable
display = pyart.graph.GridMapDisplay(grid, debug=False)


#Plot the reflectivity field between 0 to 50dBZ
display.plot_grid('reflectivity', level=0, vmin=0, vmax=50, cmap='pyart_BuDRd18')

In [ ]:
#create a figure dimension
fig = plt.figure(figsize=[10, 6])
#Plot the velocity field
display.plot_grid('velocity', level=1, vmin=-47, vmax=47, cmap='pyart_BuDRd18')

<head><h4>Assignment: <p>7. Fix the Center of the Tropical storm </p><p>8. Any variations in centre guess from Reflectivity and velocity field? </p>
    <p>9. Comment on system variation with height</p> </h4>  </head>
<p> Hint:  Parse through various levels...  </p>

### Step 6: Creating Cross sectional plots from gridded data

In [ ]:
#STUDY THE EYE Crosssection
fig = plt.figure(figsize=[15, 15])
ax1 =fig.add_subplot(211)
display.plot_latitude_slice('reflectivity', lat=19.073, vmin=0, vmax=40, ax=ax1,cmap='pyart_BuDRd18')
plt.xlim([-180, -70])
ax2 =fig.add_subplot(212)
display.plot_latitude_slice('velocity', lat=19.073, vmin=-47, vmax=47, ax=ax2,cmap='pyart_BuDRd18')
plt.xlim([-180, -70])
#ax1.set_xticks(np.arange(-180,-70,5),rotation=45);
ax1.set_xticklabels(np.arange(-180,-70,5),rotation=45);
ax2.set_xticklabels(np.arange(-180,-70,5),rotation=45);
#fig.savefig('XZ_CROSS_dBZ_19.073.png')

<head><h4>Assignment: <p>10. Comment on Eye variation with Height </p><p>11.Comment on RMR, RMV Values </p>
    <p>12. Is the system symmetric? </p> </h4>  </head>
<p> Hint:  Also verify below plot...  </p>

In [ ]:
#CREATE CROSS-SECTION PLOT ALONG A LONGITUDE
fig = plt.figure(figsize=[15, 15])
ax1 =fig.add_subplot(211)
display.plot_longitude_slice('reflectivity', lon=85.458, vmin=0, vmax=50)
plt.xlim([-80, -180])
ax1.set_xticks(np.arange(-80,-180,-5));
ax2 =fig.add_subplot(212)
display.plot_longitude_slice('velocity', lon=85.458, vmin=0, vmax=50)
plt.xlim([-80, -180])
ax2.set_xticks(np.arange(-80,-180,-5));

### Step 7: NEED FOR INTERACTIVE PLOTS

In [ ]:
radar_int = xd.io.open_iris_datatree(InputFile1).xradar.georeference()

In [ ]:
#CREATE A B_SCOPE
ref = radar_int["sweep_0"].DBZH.hvplot.quadmesh(cmap='pyart_ChaseSpectral',
                                            title='Horizontal Reflectivity (dBZ)',
                                            clim=(-20,60))
ref

In [ ]:
ref = radar_int["sweep_0"].DBZH.hvplot.quadmesh(x='x',
                                           y='y',
                                           cmap='pyart_ChaseSpectral',
                                           clabel='Horizontal Reflectivity (dBZ)',
                                           title=f'Horizontal Reflectivity \n {radar_int.attrs["instrument_name"]} Radar',
                                           clim=(-20, 60),
                                           height=500,
                                             rasterize=True,
                                           width=600,)

ref

In [ ]:
vel = radar_int["sweep_0"].VRADH.hvplot.quadmesh(x='x', y='y', cmap='pyart_BuDRd18', clabel='Radial Velocity (m/s)',
                                           title=f'Radial Velocity \n {radar_int.attrs["instrument_name"]} Radar',
                                           clim=(-50, 50),
                                           height=500,
                                             rasterize=True,
                                           width=600,)

vel

In [ ]:
#Show a combined plot
(ref + vel).cols(2)

In [ ]:
vel_hist =  radar_int["sweep_0"].VRADH.hvplot.hist()
ref_hist =  radar_int["sweep_0"].DBZH.hvplot.hist()
(vel_hist + ref_hist).cols(2)

In [ ]:
#Data filtering
radar_int["sweep_0"] = radar_int["sweep_0"].where((radar_int["sweep_0"].VRADH >= -300) & (radar_int["sweep_0"].DBZH != -32))
vel_hist =  radar_int["sweep_0"].VRADH.hvplot.hist()
ref_hist =  radar_int["sweep_0"].DBZH.hvplot.hist()
(vel_hist + ref_hist).cols(2)

In [ ]:
#Show a combined plot
refc = radar_int["sweep_0"].DBZH.hvplot.quadmesh(x='x', y='y', cmap='pyart_ChaseSpectral', title='Horizontal Reflectivity (dBZ)', clim=(-20,60), height=500, rasterize=True, width=600,)
velc = radar_int["sweep_0"].VRADH.hvplot.quadmesh(x='x',y='y', cmap='pyart_BuDRd18', clabel='Radial Velocity (m/s)',title='Radial Velocity', clim=(-50, 50), height=500,rasterize=True, width=600,)
(ref + vel + refc + velc).cols(2)

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(11, 4))
radar_int["sweep_0"].DBZH.where(radar_int["sweep_0"].DBZH >= -10).plot( x="x", y="y", cmap="ChaseSpectral", vmin=-10, vmax=50, ax=ax)
radar_int["sweep_0"].VRADH.where(radar_int["sweep_0"].VRADH >= -300).plot( x="x", y="y", cmap="pyart_BuDRd18", vmin=-50, vmax=50, ax=ax1)
ax.set_title("Filtered data plot")
ax1.set_title("")
fig.tight_layout()
plt.show()

<h1 style="color:tomato;"> <center> Any Queries??? <img src="pic/Query.gif", width=10%,height=10%> </center>   </h1>

<h1 style="color:Blue;"><center>  <img src="pic/150Yrs.jpeg", ,width=50%,height=50%> Have a Great Day... </center> </h1>
